In [1]:
!pip install pandas requests beautifulsoup4 holidays python-dateutil
import requests

In [2]:
# Get US/NYC Public Holidays

import pandas as pd
import numpy as np
import holidays

def get_nyc_holidays(start_year, end_year):
    all_holidays = []
    for year in range(start_year, end_year + 1):
        ny_holidays = holidays.UnitedStates(state='NY', years=year)
        for date, name in ny_holidays.items():
            all_holidays.append({
                'date': pd.to_datetime(date),
                'event': name,
                'type': 'Holiday'
            })
    return pd.DataFrame(all_holidays)

In [3]:
# MLB (Yankees, Mets)

def get_mlb_team_schedule(team_id, team_name, start_year, end_year):
    all_games = []
    for year in range(start_year, end_year + 1):
        url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&teamId={team_id}&season={year}&gameType=R"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            for date_info in data.get("dates", []):
                for game in date_info.get("games", []):
                    date = pd.to_datetime(game['gameDate'])
                    is_home = game['teams']['home']['team']['id'] == team_id
                    opponent = game['teams']['away']['team']['name'] if is_home else game['teams']['home']['team']['name']
                    location = "Home" if is_home else "Away"
                    all_games.append({
                        'date': date,
                        'event': f"{team_name} vs {opponent} ({location})",
                        'type': 'Sports',
                        'team': team_name,
                        'location': location
                    })
        except Exception as e:
            print(f"Failed to fetch {team_name} schedule for {year}: {e}")
    return pd.DataFrame(all_games)

In [4]:
# NBA / NHL / NFL via ESPN
from bs4 import BeautifulSoup

def get_espn_schedule(team_name, abbreviation, league, start_year, end_year):
    all_games = []
    for year in range(start_year, end_year + 1):
        url = f"https://www.espn.com/{league}/team/schedule/_/name/{abbreviation}/season/{year}"
        try:
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table')
            if not table:
                continue
            rows = table.find_all('tr')[1:]
            for row in rows:
                cols = row.find_all('td')
                if len(cols) < 2:
                    continue
                date_text = cols[0].text.strip()
                opponent = cols[1].text.strip()
                try:
                    game_date = pd.to_datetime(f"{date_text} {year}", errors='coerce')
                    if pd.isnull(game_date):
                        continue
                except:
                    continue
                location = "Away" if "@" in opponent else "Home"
                opponent_clean = opponent.replace("@", "").replace("vs", "").strip()
                all_games.append({
                    'date': game_date,
                    'event': f"{team_name} vs {opponent_clean} ({location})",
                    'type': 'Sports',
                    'team': team_name,
                    'location': location
                })
        except Exception as e:
            print(f"Failed to fetch {team_name} schedule for {year}: {e}")
    return pd.DataFrame(all_games)


In [5]:
# Fixes issues with date times not being insync

def localize_to_nyc(df):
    df['date'] = pd.to_datetime(df['date'])
    if df['date'].dt.tz is None or df['date'].dt.tz is pd.NaT:
        df['date'] = df['date'].dt.tz_localize('America/New_York')
    else:
        df['date'] = df['date'].dt.tz_convert('America/New_York')
    return df

In [6]:
# Combine All

def get_all_event_data(start_year=2020, end_year=2025):
    # Holidays
    holidays_df = get_nyc_holidays(start_year, end_year)

    # MLB Teams (Yankees, Mets)
    yankees_df = get_mlb_team_schedule(team_id=147, team_name='Yankees', start_year=start_year, end_year=end_year)
    mets_df = get_mlb_team_schedule(team_id=121, team_name='Mets', start_year=start_year, end_year=end_year)

    # NBA, NFL, NHL via ESPN (approximate dates only, times not available)
    knicks_df = get_espn_schedule('Knicks', 'ny', 'nba', start_year, end_year)
    nets_df = get_espn_schedule('Nets', 'bkn', 'nba', start_year, end_year)
    giants_df = get_espn_schedule('Giants', 'nyg', 'nfl', start_year, end_year)
    jets_df = get_espn_schedule('Jets', 'nyj', 'nfl', start_year, end_year)
    rangers_df = get_espn_schedule('Rangers', 'nyr', 'nhl', start_year, end_year)

    # Combine
    # Standardize to NYC timezone
    dfs = [
        localize_to_nyc(holidays_df),
        localize_to_nyc(yankees_df),
        localize_to_nyc(mets_df),
        localize_to_nyc(knicks_df),
        localize_to_nyc(nets_df),
        localize_to_nyc(giants_df),
        localize_to_nyc(jets_df),
        localize_to_nyc(rangers_df)
    ]

    combined_df = pd.concat(dfs).sort_values('date').reset_index(drop=True)

    return combined_df

In [7]:
# Run and save
events_df = get_all_event_data()
print(events_df.head(100))

                        date                                    event  \
0  2020-01-01 00:00:00-05:00                           New Year's Day   
1  2020-01-01 00:00:00-05:00                Knicks vs Portland (Home)   
2  2020-01-01 00:00:00-05:00             Giants vs Thu, Aug 13 (Home)   
3  2020-01-01 00:00:00-05:00               Jets vs Sun, Sep 13 (Home)   
4  2020-01-01 00:00:00-05:00             Giants vs Mon, Sep 14 (Home)   
..                       ...                                      ...   
95 2020-08-05 18:05:00-04:00      Mets vs Washington Nationals (Away)   
96 2020-08-05 19:35:00-04:00      Yankees vs Baltimore Orioles (Away)   
97 2020-08-06 18:05:00-04:00  Yankees vs Philadelphia Phillies (Away)   
98 2020-08-07 18:40:00-04:00         Yankees vs Tampa Bay Rays (Away)   
99 2020-08-07 19:10:00-04:00             Mets vs Miami Marlins (Home)   

       type     team location  
0   Holiday      NaN      NaN  
1    Sports   Knicks     Home  
2    Sports   Giants     Ho

In [10]:
# Adding day type features
events_df['year'] = events_df['date'].dt.year
events_df['month'] = events_df['date'].dt.month
events_df['day'] = events_df['date'].dt.day
events_df['day_of_week'] = events_df['date'].dt.dayofweek
events_df['day_name'] = events_df['date'].dt.day_name()
events_df['is_weekend'] = events_df['day_of_week'].isin([5, 6])
events_df['start_hour'] = events_df['date'].dt.hour

print(events_df.head())

                       date                         event     type    team  \
0 2020-01-01 00:00:00-05:00                New Year's Day  Holiday     NaN   
1 2020-01-01 00:00:00-05:00     Knicks vs Portland (Home)   Sports  Knicks   
2 2020-01-01 00:00:00-05:00  Giants vs Thu, Aug 13 (Home)   Sports  Giants   
3 2020-01-01 00:00:00-05:00    Jets vs Sun, Sep 13 (Home)   Sports    Jets   
4 2020-01-01 00:00:00-05:00  Giants vs Mon, Sep 14 (Home)   Sports  Giants   

  location  year  month  day  day_of_week   day_name  is_weekend  start_hour  
0      NaN  2020      1    1            2  Wednesday       False           0  
1     Home  2020      1    1            2  Wednesday       False           0  
2     Home  2020      1    1            2  Wednesday       False           0  
3     Home  2020      1    1            2  Wednesday       False           0  
4     Home  2020      1    1            2  Wednesday       False           0  


In [16]:
# Clean 'event' column by stripping tabs and excessive spaces

# Step 1: Get all events
events_df = get_all_event_data()

# Step 2: Filter out minor holidays
major_holidays = {
    "New Year's Day",
    "Martin Luther King Jr. Day",
    "Presidents' Day",
    "Memorial Day",
    "Independence Day",
    "Labor Day",
    "Columbus Day",
    "Veterans Day",
    "Thanksgiving",
    "Thanksgiving Day",
    "Christmas Day",
    "Washington's Birthday",
    "Juneteenth National Independence Day"
}

# Remove holidays not in the major list
events_df = events_df[~((events_df['type'] == 'Holiday') & (~events_df['event'].isin(major_holidays)))]

# Step 3: Proceed with time-based feature engineering
events_df['year'] = events_df['date'].dt.year
events_df['month'] = events_df['date'].dt.month
events_df['day'] = events_df['date'].dt.day
events_df['day_of_week'] = events_df['date'].dt.dayofweek
events_df['day_name'] = events_df['date'].dt.day_name()
events_df['is_weekend'] = events_df['day_of_week'].isin([5, 6])
events_df['start_hour'] = events_df['date'].dt.hour  # Add this too

# Step 4: Save or preview
print(events_df.head())


                       date                         event     type    team  \
0 2020-01-01 00:00:00-05:00                New Year's Day  Holiday     NaN   
1 2020-01-01 00:00:00-05:00     Knicks vs Portland (Home)   Sports  Knicks   
2 2020-01-01 00:00:00-05:00  Giants vs Thu, Aug 13 (Home)   Sports  Giants   
3 2020-01-01 00:00:00-05:00    Jets vs Sun, Sep 13 (Home)   Sports    Jets   
4 2020-01-01 00:00:00-05:00  Giants vs Mon, Sep 14 (Home)   Sports  Giants   

  location  year  month  day  day_of_week   day_name  is_weekend  start_hour  
0      NaN  2020      1    1            2  Wednesday       False           0  
1     Home  2020      1    1            2  Wednesday       False           0  
2     Home  2020      1    1            2  Wednesday       False           0  
3     Home  2020      1    1            2  Wednesday       False           0  
4     Home  2020      1    1            2  Wednesday       False           0  


In [18]:
# Only adjust sports events that have a start hour of 0
sports_with_midnight_start = (events_df['type'] == 'Sports') & (events_df['date'].dt.hour == 0)

# Add 19 hours (7 PM) to those entries
events_df.loc[sports_with_midnight_start, 'date'] = events_df.loc[sports_with_midnight_start, 'date'] + pd.Timedelta(hours=19)

# Recalculate start_hour
events_df['start_hour'] = events_df['date'].dt.hour


In [28]:
from google.colab import drive
drive.mount('/content/drive')

import os

output_folder = "/content/drive/MyDrive/NYC_Subway_Analysis"
os.makedirs(output_folder, exist_ok=True)

events_df.to_csv(f"{output_folder}/nyc_events_2020_2025.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#Prepare events DataFrame for hourly merging

def create_hourly_event_flags(events_df, start_year=2020, end_year=2025):
    start = pd.Timestamp(f"{start_year}-01-01 00:00:00", tz='America/New_York')
    end = pd.Timestamp(f"{end_year}-12-31 23:00:00", tz='America/New_York')
    all_hours = pd.date_range(start=start, end=end, freq='h')

    hourly_events = pd.DataFrame(index=all_hours)

    hourly_events['is_holiday'] = 0
    hourly_events['is_home_sporting_event'] = 0
    hourly_events['is_away_sporting_event'] = 0
    hourly_events['event_names'] = ""

    # Flag holidays
    holidays = events_df[events_df['type'] == 'Holiday']
    for idx, row in holidays.iterrows():
        day = row['date'].date()
        day_hours = pd.date_range(start=pd.Timestamp(day, tz='America/New_York'),
                                  end=pd.Timestamp(day, tz='America/New_York') + pd.Timedelta(hours=23),
                                  freq='h')
        hourly_events.loc[hourly_events.index.isin(day_hours), 'is_holiday'] = 1
        for h in day_hours:
            if hourly_events.at[h, 'event_names']:
                hourly_events.at[h, 'event_names'] += "; " + row['event']
            else:
                hourly_events.at[h, 'event_names'] = row['event']

    # Flag sports events by location (home/away)
    sports = events_df[events_df['type'] == 'Sports']
    for idx, row in sports.iterrows():
        event_start = row['date']
        window_hours = pd.date_range(start=event_start - pd.Timedelta(hours=2),
                                     end=event_start + pd.Timedelta(hours=3),
                                     freq='h')
        window_hours = [h for h in window_hours if h in hourly_events.index]

        # Use 'location' column
        loc = str(row.get('location', '')).lower()
        if loc == 'home':
            col_flag = 'is_home_sporting_event'
        elif loc == 'away':
            col_flag = 'is_away_sporting_event'
        else:
            # If location is missing or unexpected, skip or set neither flag
            continue

        hourly_events.loc[window_hours, col_flag] = 1

        for h in window_hours:
            if hourly_events.at[h, 'event_names']:
                hourly_events.at[h, 'event_names'] += "; " + row['event']
            else:
                hourly_events.at[h, 'event_names'] = row['event']

    return hourly_events.reset_index().rename(columns={'index': 'timestamp'})


# Usage:
hourly_events_df = create_hourly_event_flags(events_df)
print(hourly_events_df.head(30))

                   timestamp  is_holiday  is_home_sporting_event  \
0  2020-01-01 00:00:00-05:00           1                       0   
1  2020-01-01 01:00:00-05:00           1                       0   
2  2020-01-01 02:00:00-05:00           1                       0   
3  2020-01-01 03:00:00-05:00           1                       0   
4  2020-01-01 04:00:00-05:00           1                       0   
5  2020-01-01 05:00:00-05:00           1                       0   
6  2020-01-01 06:00:00-05:00           1                       0   
7  2020-01-01 07:00:00-05:00           1                       0   
8  2020-01-01 08:00:00-05:00           1                       0   
9  2020-01-01 09:00:00-05:00           1                       0   
10 2020-01-01 10:00:00-05:00           1                       0   
11 2020-01-01 11:00:00-05:00           1                       0   
12 2020-01-01 12:00:00-05:00           1                       0   
13 2020-01-01 13:00:00-05:00           1        

In [29]:
from google.colab import drive
drive.mount('/content/drive')

import os

output_folder = "/content/drive/MyDrive/NYC_Subway_Analysis"
os.makedirs(output_folder, exist_ok=True)

hourly_events_df.to_csv(f"{output_folder}/nyc_events_hourly_2020_2025.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
